In [10]:
import datetime
import time
import os
import pandas as pd
import json
totalRuns=3
endpoint = "http://localhost:9090/api/v1/query?query="
total_requests_query = f"{endpoint}sum(istio_requests_total{{namespace=~'sock-shop|hawk-ns'}})[1h:1s]"
total_requests_collector_query = f"{endpoint}sum(istio_requests_total{{namespace=~'sock-shop|hawk-ns',destination_service_name='collector'}})[1h:1s]"
latency_ms_collector_query = f"{endpoint}(sum(increase(istio_request_duration_milliseconds_sum{{destination_app='collector'}}[1h]))/sum(increase(istio_request_duration_milliseconds_count{{destination_app='collector'}}[1h])))[1h:1s]"

def get_prometheus_query(r,download_filename,first,last,query)->pd.Series:
    filename = f"testresults/{r}_{download_filename}"
    os.system(f'curl -o {filename} -g "{query}"')
    print(query)
    with open(filename, 'r') as f:
        data = json.load(f)["data"]["result"][0]["values"]
    df = pd.DataFrame(data).convert_dtypes().set_index(0,drop=True)
    
    return pd.to_numeric(df.loc[first:last][1])

In [14]:
for r in range(totalRuns):
    print(f"Run: {r}/{totalRuns}")
    os.system(f"locust --csv=testresults/{r}")
    hist_df = pd.read_csv(f"testresults/{r}_stats_history.csv")
    first = int(hist_df["Timestamp"].head(1).values[0])
    last = int(hist_df["Timestamp"].tail(1).values[0])
    hist_df.set_index("Timestamp",inplace=True)
    print("Sleep 10 seconds...")

Run: 0/3


[2023-03-03 10:21:25,222] ciph/WARNING/locust.main: System open file limit '8192' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2023-03-03 10:21:25,223] ciph/INFO/locust.main: Run time limit set to 61 seconds
[2023-03-03 10:21:25,223] ciph/INFO/locust.main: Starting Locust 2.15.0
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated              

Sleep 10 seconds...
Run: 1/3


[2023-03-03 10:22:28,504] ciph/WARNING/locust.main: System open file limit '8192' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2023-03-03 10:22:28,512] ciph/INFO/locust.main: Run time limit set to 61 seconds
[2023-03-03 10:22:28,513] ciph/INFO/locust.main: Starting Locust 2.15.0
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated              

Sleep 10 seconds...
Run: 2/3


[2023-03-03 10:23:30,316] ciph/WARNING/locust.main: System open file limit '8192' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2023-03-03 10:23:30,318] ciph/INFO/locust.main: Run time limit set to 61 seconds
[2023-03-03 10:23:30,319] ciph/INFO/locust.main: Starting Locust 2.15.0
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated              

Sleep 10 seconds...


In [6]:
hist_df = pd.read_csv(f"testresults/{0}_stats_history.csv")
first = int(hist_df["Timestamp"].head(1).values[0])
last = int(hist_df["Timestamp"].tail(1).values[0])
hist_df.set_index("Timestamp",inplace=True)

In [7]:
hist_df["Istio_Requests_Total"] = get_prometheus_query(0,"istio_total_requests.json",first,last,total_requests_query)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18865    0 18865    0     0   839k      0 --:--:-- --:--:-- --:--:--  877k


In [11]:
hist_df["Istio_Requests_Collector_Total"] = get_prometheus_query(0,"istio_total_requests_collector.json",first,last,total_requests_collector_query)

http://localhost:9090/api/v1/query?query=sum(istio_requests_total{namespace=~'sock-shop|hawk-ns',destination_service_name='collector'})[1h:1s]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    63  100    63    0     0   3628      0 --:--:-- --:--:-- --:--:--  3937


IndexError: list index out of range